# F1 Data Analysis

idee:
- pulire i dati dei giri trovando un modo di valutare gli errori (tempi che oscillano possno essere dovuti al graining)


In [ ]:
import fastf1

session = fastf1.get_session(2025, 'Australia','FP1')
session.load()

In [ ]:
session.laps.pick_driver('NOR')

c:\Users\marti\Desktop\programmazione\F1\.venv\lib\site-packages\fastf1\core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


21   0 days 00:16:32.768000
22   0 days 00:17:54.664000
23   0 days 00:19:56.297000
24   0 days 00:21:15.062000
25   0 days 00:23:23.487000
26   0 days 00:24:41.740000
27   0 days 00:39:12.338000
28   0 days 00:41:36.009000
29   0 days 00:42:58.791000
30   0 days 00:44:52.902000
31   0 days 00:46:11.149000
32   0 days 01:02:56.640000
33   0 days 01:05:20.247000
34   0 days 01:06:56.601000
35   0 days 01:08:14.119000
36   0 days 01:10:13.269000
37   0 days 01:12:02.856000
38   0 days 01:13:20.108000
39   0 days 01:14:44.079000
40   0 days 01:17:15.616000
41   0 days 01:18:50.654000
Name: Time, dtype: timedelta64[ns]